# Checking and describing the generated data

It is always beneficial to add a notebook that quickly looks into the data to help you remember, which data you collected and if it actually looks correct.

In [59]:
from algbench import describe, read_as_pandas, Benchmark
from _conf import EXPERIMENT_DATA

In [60]:
describe(EXPERIMENT_DATA)

An entry in the database can look like this:
_____________________________________________
 result:
| num_nodes: 25
| lower_bound: 118462718.0
| objective: 118462718.0
 timestamp: 2023-11-16T18:36:05.040082
 runtime: 0.022690534591674805
 stdout: [[0.0005812644958496094, 'Set parameter Username\n'], [0.0014045238494873047,...
 stderr: []
 logging: [{'name': 'Evaluation', 'msg': 'Building model.', 'args': [], 'levelname': 'I...
 env_fingerprint: 242c13fb490e94b846e75f6af67c4a81ed26705e
 args_fingerprint: a86db1b3196b76d0b33a446f9d3f01767976b611
 parameters:
| func: run_solver
| args:
|| instance_name: random_euclidean_25_0
|| time_limit: 90
|| strategy: GurobiTspSolver
|| opt_tol: 0.001
 argv: ['01_run_benchmark.py']
 env:
| hostname: workstation-r7
| python_version: 3.10.12 (main, Jul  5 2023, 18:54:27) [GCC 11.2.0]
| python: /home/krupke/anaconda3/envs/mo310/bin/python3
| cwd: /home/krupke/Repositories/cpsat-primer/examples/tsp_evaluation
| git_revision: afa10479f9816f624ea8759ecdb4b3

In [61]:
t = read_as_pandas(
    EXPERIMENT_DATA,
    lambda entry: {
        "instance_name": entry["parameters"]["args"]["instance_name"],
        "num_nodes": entry["result"]["num_nodes"],
        "time_limit": entry["parameters"]["args"]["time_limit"],
        "strategy": entry["parameters"]["args"]["strategy"],
        "opt_tol": entry["parameters"]["args"]["opt_tol"],
        "runtime": entry["runtime"],
        "objective": entry["result"]["objective"],
        "lower_bound": entry["result"]["lower_bound"],
    },
)
t.drop_duplicates(inplace=True, subset=["instance_name", "num_nodes", "strategy"])
t["opt_gap"] = (t["objective"] - t["lower_bound"]) / t["lower_bound"]
t.sort_values(["num_nodes", "instance_name"])

,instance_name,num_nodes,time_limit,strategy,opt_tol,runtime,objective,lower_bound,opt_gap
0,random_euclidean_25_0,25,90,GurobiTspSolver,0.001,0.022691,118462718.0,118462718.0,0.000000
5,random_euclidean_25_0,25,90,CpSatTspSolverV1,0.001,0.062708,118462718.0,118462718.0,0.000000
10,random_euclidean_25_0,25,90,CpSatTspSolverDantzig,0.001,0.128039,118462718.0,118462718.0,0.000000
15,random_euclidean_25_0,25,90,CpSatTspSolverMtz,0.001,0.139193,118462718.0,118462718.0,0.000000
20,random_euclidean_25_1,25,90,GurobiTspSolver,0.001,0.014724,80173028.0,80173028.0,0.000000
...,...,...,...,...,...,...,...,...,...
380,random_euclidean_75_1,75,90,CpSatTspSolverV1,0.001,9.720901,75684600.0,75620879.0,0.000843
385,random_euclidean_75_1,75,90,CpSatTspSolverDantzig,0.001,90.301121,75684600.0,75684600.0,0.000000
314,random_euclidean_75_2,75,90,GurobiTspSolver,0.001,0.230614,81191992.0,81191992.0,0.000000
319,random_euclidean_75_2,75,90,CpSatTspSolverV1,0.001,2.195840,81191992.0,81191992.0,0.000000


In [62]:
for entry in Benchmark(EXPERIMENT_DATA):
    
    if entry["parameters"]["args"]["instance_name"] == "random_euclidean_100_1" and entry["parameters"]["args"]["strategy"] == "GurobiTspSolver":
        print("=====================================")
        stdout = "".join( e[1] for e in entry["stdout"])
        stderr = "".join( e[1] for e in entry["stderr"])
        print(stdout)
        print(stderr)
        if not stdout.strip():
            print("No stdout")
        print("=====================================")

## Check for errors in the data

You always want to check if the results you got are actually feasible. Errors easily happen and are not always visible on the plots.
Thus, you want to do some basic checks to detect errors early on. For example, you could accidentally have swapped lower and upper bounds in the data generation process.
Depending on your plots, this may not be visible, and you may end up comparing the wrong data and draw the wrong conclusions.
Or, you could have accidentally swapped runtime and objective values, which could look reasonable in the data as the runtime and the objective often increase with the instance size.

A very basic check is to check if the best lower and upper bounds do not contradict each other. Many errors will be caught by this check. However, you often need some tolerance to account for numerical errors.

In [63]:
assert (t.dropna()["opt_gap"]>=-0.0001).all(), "Optimality gap is negative!"

In [64]:
# Always make sure that your results are not trivially wrong
#  - e.g. lower bound is higher than objective
max_lb = t.groupby(["instance_name"])["lower_bound"].max()
min_obj = t.groupby(["instance_name"])["objective"].min()
eps = 0.0001  # some tolerance is needed when working with floats.
bad_instances = max_lb[max_lb-min_obj>eps*max_lb].index.to_list()
from IPython.display import display
display(t[t["instance_name"].isin(bad_instances)])
assert len(bad_instances) == 0, "Bad instances detected: {}".format(bad_instances)

,instance_name,num_nodes,time_limit,strategy,opt_tol,runtime,objective,lower_bound,opt_gap
